In [3]:
pip install python-dotenv

In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# Importações
import os
import cv2
import pandas as pd
import math
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, InputLayer, Dropout
from skimage.transform import resize
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from matplotlib import pyplot as plt
from dotenv import load_dotenv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Carregar variáveis de ambiente
load_dotenv('amb_var.env')

False

In [5]:
import tensorflow as tf

# Verificar se o GPU está disponível
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print('GPU disponível: ', gpus)
else:
    print('Nenhum GPU disponível')

GPU disponível:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
# Separação de frames do vídeo de treinamento
count = 0
videoFile = r'/content/drive/MyDrive/Colab Notebooks/Acidentes-ICity Train - Made with Clipchamp.mp4'
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5)  # frame rate
while(cap.isOpened()):
    frameId = cap.get(1)  # current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename = "%d.jpg" % count
        count += 1
        cv2.imwrite(filename, frame)
cap.release()

In [10]:
# Leitura do CSV de classificação
data = pd.read_csv('mapping_final.csv')
X = [plt.imread('' + img_name) for img_name in data.Image_ID]
X = np.array(X)

y = data.Class
dummy_y = to_categorical(y)

In [11]:
# Preprocessamento das imagens
X = [resize(img, preserve_range=True, output_shape=(224, 224)).astype(int) for img in X]
X = np.array(X)
X = preprocess_input(X, data_format=None)

In [12]:
# Divisão em treino e teste
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

In [15]:
batch_size = 32
num_batches = int(np.ceil(X_train.shape[0] / batch_size))
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))



# Inicializa X_train_transformed
X_train_transformed = np.zeros((X_train.shape[0], 7 * 7 * 512))

for i in range(num_batches):
    start = i * batch_size
    end = (i + 1) * batch_size
    X_train_batch = X_train[start:end]
    X_train_batch_transformed = base_model.predict(datagen.flow(X_train_batch, batch_size=batch_size, shuffle=False), steps=np.ceil(X_train_batch.shape[0]/batch_size))
    X_train_batch_transformed = X_train_batch_transformed.reshape(X_train_batch_transformed.shape[0], 7 * 7 * 512)
    X_train_transformed[start:end] = X_train_batch_transformed

X_train = X_train_transformed
X_valid = base_model.predict(X_valid)
X_valid = X_valid.reshape(X_valid.shape[0], 7 * 7 * 512)

train = X_train / X_train.max()
X_valid = X_valid / X_train.max()

model = Sequential()
model.add(InputLayer((7 * 7 * 512,)))
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train, y_train, batch_size=32, epochs=100, validation_data=(X_valid, y_valid), callbacks=[early_stop])

model.save('MODELGPU')


44/44 [==============================] - 2s 34ms/step
Epoch 1/100
102/102 [==============================] - 3s 9ms/step - loss: 0.5954 - accuracy: 0.7012 - val_loss: 0.5708 - val_accuracy: 0.7170
Epoch 2/100
102/102 [==============================] - 1s 5ms/step - loss: 0.3964 - accuracy: 0.8283 - val_loss: 0.5725 - val_accuracy: 0.7464
Epoch 3/100
102/102 [==============================] - 1s 5ms/step - loss: 0.2154 - accuracy: 0.9155 - val_loss: 0.6775 - val_accuracy: 0.7443
Epoch 4/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0951 - accuracy: 0.9751 - val_loss: 0.7904 - val_accuracy: 0.7393
Epoch 5/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0455 - accuracy: 0.9874 - val_loss: 0.9093 - val_accuracy: 0.7471
Epoch 6/100
102/102 [==============================] - 0s 5ms/step - loss: 0.0229 - accuracy: 0.9966 - val_loss: 1.0886 - val_accuracy: 0.7393
Epoch 7/100
102/102 [==============================] - 1s 5ms/step - loss: 0.0135 - accu

In [22]:
# Teste
count = 0
videoFile = "/content/Accident-1.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5)  # frame rate
while(cap.isOpened()):
    frameId = cap.get(1)  # current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename = "test%d.jpg" % count
        count += 1
        cv2.imwrite(filename, frame)
cap.release()

In [23]:
# Leitura de CSV de teste e preprocessamento
test = pd.read_csv('test.csv')
test_image = [plt.imread('' + img_name) for img_name in test.Image_ID]
test_img = np.array(test_image)
test_image = [resize(img, preserve_range=True, output_shape=(224, 224)).astype(int) for img in test_img]
test_image = preprocess_input(np.array(test_image), data_format=None)
test_image = base_model.predict(test_image)
test_image = test_image.reshape(test_image.shape[0], 7 * 7 * 512)
test_image = test_image / test_image.max()

predictions = model.predict(test_image)

1/1 [==============================] - 0s 48ms/step


In [24]:
# Teste em tempo real
cap = cv2.VideoCapture('/content/Untitled video - Made with Clipchamp.mp4')
i = 0
flag = 0
while(True):
    ret, frame = cap.read()
    if ret == True:
        if i % 5 == 0:  # Predição a cada 5 frames
            if predictions[int(i / 15) % 9][0] < predictions[int(i / 15) % 9][1]:
                predict = "Sem Acidente"
            else:
                predict = "Acidente"
                flag = 1
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, predict, (50, 50), font, 1, (0, 255, 255), 3, cv2.LINE_4)
        cv2.imshow('Frame', frame)
        i += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
if flag == 1:
    print('Acidente')

cap.release()
cv2.destroyAllWindows()


DisabledFunctionError: ignored

In [25]:
!zip -r /content/nome_da_pasta.zip /content/MODELGPU


  adding: content/MODELGPU/ (stored 0%)
  adding: content/MODELGPU/saved_model.pb (deflated 87%)
  adding: content/MODELGPU/keras_metadata.pb (deflated 84%)
  adding: content/MODELGPU/variables/ (stored 0%)
  adding: content/MODELGPU/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: content/MODELGPU/variables/variables.index (deflated 57%)
  adding: content/MODELGPU/fingerprint.pb (stored 0%)
  adding: content/MODELGPU/assets/ (stored 0%)


In [27]:
from google.colab import files
files.download("/content/MODELGPU.zip")


FileNotFoundError: ignored

In [20]:
def apagar_imagens():
    for i in range(5051):
        nome_arquivo = f"{i}.jpg"
        if os.path.exists(nome_arquivo):
            os.remove(nome_arquivo)
            print(f"Arquivo {nome_arquivo} apagado.")
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

    for i in range(9):
        nome_arquivo = f"test{i}.jpg"
        if os.path.exists(nome_arquivo):
            os.remove(nome_arquivo)
            print(f"Arquivo {nome_arquivo} apagado.")
        else:
            print(f"Arquivo {nome_arquivo} não encontrado.")

delete = input('''Digite D para apagar as fotos gerados:
Digite N para manter as fotos: ''').upper()

if delete == 'D':
    apagar_imagens()
else:
    print('Imagens Mantidas')

A saída de streaming foi truncada nas últimas 5000 linhas.
Arquivo 60.jpg não encontrado.
Arquivo 61.jpg não encontrado.
Arquivo 62.jpg não encontrado.
Arquivo 63.jpg não encontrado.
Arquivo 64.jpg não encontrado.
Arquivo 65.jpg não encontrado.
Arquivo 66.jpg não encontrado.
Arquivo 67.jpg não encontrado.
Arquivo 68.jpg não encontrado.
Arquivo 69.jpg não encontrado.
Arquivo 70.jpg não encontrado.
Arquivo 71.jpg não encontrado.
Arquivo 72.jpg não encontrado.
Arquivo 73.jpg não encontrado.
Arquivo 74.jpg não encontrado.
Arquivo 75.jpg não encontrado.
Arquivo 76.jpg não encontrado.
Arquivo 77.jpg não encontrado.
Arquivo 78.jpg não encontrado.
Arquivo 79.jpg não encontrado.
Arquivo 80.jpg não encontrado.
Arquivo 81.jpg não encontrado.
Arquivo 82.jpg não encontrado.
Arquivo 83.jpg não encontrado.
Arquivo 84.jpg não encontrado.
Arquivo 85.jpg não encontrado.
Arquivo 86.jpg não encontrado.
Arquivo 87.jpg não encontrado.
Arquivo 88.jpg não encontrado.
Arquivo 89.jpg não encontrado.
Arquivo 90.